In [41]:
# Se importa vectorizer de sklearn
import numpy as np 
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
# abrimos el datafrem de juegos
df=pd.read_csv(r'C:\Users\p2_ge\Documents\proyectos\PI-MLOPS\Juegos_steam.csv')

In [43]:
df.head()

,Id_juego,Nombre,Desarrollador,Año_lanzamiento,Género
0,761140,Lost Summoner Kitty,Kotoshiro,2018,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,Ironbound,Secret Level SRL,2018,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,Real Pool 3D - Poolians,Poolians.com,2017,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,弹炸人2222,彼岸领域,2017,"['Action', 'Adventure', 'Casual']"
4,773570,Log Challenge,N/D,2018,"Action, Casual, Indie, Sports"


In [44]:
#pasamos la columna de Género a un formato de string, para que get dummies reconozca los generos únicos
def norm(y):
    p=''
    l=y.split(',')
    for a in l:
        a=a.strip("[]' ")
        p=p+a+', '
    p.strip(', ')
    return p


df.Género=df.Género.apply(lambda x: norm(x))

In [45]:
#buscamos que no haya nulos o duplicados, y los borramos si es que existen
print('duplicados: ',df.duplicated().sum())
df.drop_duplicates(inplace=True)
print('nulos: ',df.duplicated().sum().sum())

duplicados:  3
nulos:  0


In [46]:
# Obtenemos los dummies de Genero
X=df.Género.str.get_dummies(', ')

In [47]:
# ponemos los id de los juegos como indices de los dummies
X=X.set_index(pd.Index(df.Id_juego))

In [48]:
X

,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
Id_juego,,,,,,,,,,,,,,,,,,,,,
761140,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
643980,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
670290,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
767400,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
773570,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773640,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
733530,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
610660,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [49]:
# se aplica la similitud del coseno a nuestros vectores
similitud = cosine_similarity(X)

In [50]:
similitudes=pd.DataFrame(similitud,index=df.Id_juego,columns=df.Id_juego)

In [51]:
similitudes

Id_juego,761140,643980,670290,767400,773570,772540,774276,774277,774278,768800,...,771810,767590,747320,769330,745400,773640,733530,610660,658870,681550
Id_juego,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.447214,0.600000,0.516398,0.670820,0.516398,0.447214,0.447214,0.447214,0.670820,...,0.516398,0.632456,0.316228,0.632456,0.670820,0.894427,0.774597,0.516398,0.632456,0.600000
643980,0.447214,1.000000,0.447214,0.000000,0.250000,0.000000,0.500000,0.500000,0.500000,0.250000,...,0.288675,0.353553,0.707107,0.353553,0.250000,0.500000,0.577350,0.288675,0.353553,0.223607
670290,0.600000,0.447214,1.000000,0.258199,0.670820,0.258199,0.894427,0.894427,0.894427,0.670820,...,0.258199,0.632456,0.316228,0.632456,0.447214,0.670820,0.516398,0.516398,0.632456,0.400000
767400,0.516398,0.000000,0.258199,1.000000,0.577350,0.666667,0.000000,0.000000,0.000000,0.288675,...,0.666667,0.408248,0.000000,0.408248,0.866025,0.288675,0.333333,0.000000,0.408248,0.516398
773570,0.670820,0.250000,0.670820,0.577350,1.000000,0.288675,0.500000,0.500000,0.500000,0.500000,...,0.577350,0.707107,0.353553,0.707107,0.750000,0.500000,0.577350,0.288675,0.707107,0.447214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773640,0.894427,0.500000,0.670820,0.288675,0.500000,0.288675,0.500000,0.500000,0.500000,0.750000,...,0.288675,0.707107,0.353553,0.707107,0.500000,1.000000,0.866025,0.577350,0.707107,0.447214
733530,0.774597,0.577350,0.516398,0.333333,0.577350,0.000000,0.288675,0.288675,0.288675,0.577350,...,0.333333,0.816497,0.408248,0.816497,0.577350,0.866025,1.000000,0.333333,0.816497,0.258199
610660,0.516398,0.288675,0.516398,0.000000,0.288675,0.333333,0.577350,0.577350,0.577350,0.866025,...,0.333333,0.408248,0.408248,0.408248,0.288675,0.577350,0.333333,1.000000,0.408248,0.516398


In [52]:
#sacamos el id_del juego y el nombre a un dtaaframe donde se guardarán los datos de la función
dffuncion=df[['Id_juego','Nombre']]

In [53]:
#creamos una función que saca el top 5 de juesgos recomendados para cada juego
def nombres(id):
    nombres=similitudes.loc[id].nlargest(6)[1:6].index.to_list()
    l=df['Nombre'][df['Id_juego'].isin(nombres)].to_list()
    return l

In [ ]:
#se aplica la función al dataframe final, crenado la columna donde estarán los resultados
dffuncion.loc[:,'juegos']=df.Id_juego.apply(lambda x:nombres(x)).copy()

In [55]:
dffuncion

,Id_juego,Nombre,juegos
0,761140,Lost Summoner Kitty,"[Pixel Puzzles 2: Anime, World of Cinema - Dir..."
1,643980,Ironbound,"[Shadow Hunter, Immortal Empire, Immortal Empi..."
2,670290,Real Pool 3D - Poolians,"[Pixel Puzzles Ultimate - Puzzle Pack: Rio, Pi..."
3,767400,弹炸人2222,"[Atomic Adam: Episode 1, Biozone, Luxor: 5th P..."
4,773570,Log Challenge,"[Canyon Capers - Rio Fever, Lighthockey, 12 or..."
...,...,...,...
32130,773640,Colony On Mars,"[Fate of the World: Tipping Point, Fate of the..."
32131,733530,LOGistICAL: South Africa,"[Puzzler World 2, iBomber Defense Pacific, Bum..."
32132,610660,Russian Roads,"[City Car Driving, Try Hard Parking, Car Mecha..."
32133,658870,EXIT 2 - Directions,"[Zen of Sudoku, Gumboy - Crazy Adventures™, RI..."


In [56]:
#guardamos en un csv el dataframe final para la función
dffuncion.to_csv('consulta_modelo1.csv', index=False)

In [57]:
dffuncion.head()

,Id_juego,Nombre,juegos
0,761140,Lost Summoner Kitty,"[Pixel Puzzles 2: Anime, World of Cinema - Dir..."
1,643980,Ironbound,"[Shadow Hunter, Immortal Empire, Immortal Empi..."
2,670290,Real Pool 3D - Poolians,"[Pixel Puzzles Ultimate - Puzzle Pack: Rio, Pi..."
3,767400,弹炸人2222,"[Atomic Adam: Episode 1, Biozone, Luxor: 5th P..."
4,773570,Log Challenge,"[Canyon Capers - Rio Fever, Lighthockey, 12 or..."


In [58]:
def recomendacion_juego( id_de_producto ):
    return dffuncion.juegos[dffuncion.Id_juego==id_de_producto]


In [59]:
recomendacion_juego(773570)

4    [Canyon Capers - Rio Fever, Lighthockey, 12 or...
Name: juegos, dtype: object